# Input source data and libraries

In [1]:
# Input libraries
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
import pandapower as pp

In [2]:
# Input MEF data
MEF = pd.read_csv("./source_data/CA_LR_24h.csv",index_col = 0)['MEF'].tolist()

In [3]:
# Input EV charging data from a workplace in the whole year of 2019 with a certain number of charging piles

num_pile = 50 # The available numbers are 10, 50, 90, 130, 170, 210, 250, 290, 330, 370, 410, and 450

EVC_workplace_annual_50 = pd.read_csv("./result/EVC/EVC_workplace_50.csv",index_col=0)

In [23]:
# based load of a workplace building with 30 floors from 2019
BL_workplace_avg = pd.read_csv("./result/base_load/BL_workplace_annual_avg.csv",index_col=[0,1])
BL_workplace_25th = pd.read_csv("./result/base_load/BL_workplace_annual_25th.csv",index_col=[0,1])
BL_workplace_75th = pd.read_csv("./result/base_load/BL_workplace_annual_75th.csv",index_col=[0,1])
BL_workplace_2019_avg = BL_workplace_avg.loc[BL_workplace_avg.index.get_level_values(0) == 2019]
BL_workplace_2019_25th = BL_workplace_25th.loc[BL_workplace_25th.index.get_level_values(0) == 2019]
BL_workplace_2019_75th = BL_workplace_75th.loc[BL_workplace_75th.index.get_level_values(0) == 2019]

# The function to extract the EV charging session data for one day

In [4]:
# Effective dates on the EV charging dataset
# Label the working days and non-working days in 2019 in USA

start = datetime(2019,1,1)
end = datetime(2019,12,31)
weekday_index = pd.bdate_range(start, end)
weekday_list = weekday_index.strftime("%Y-%m-%d").tolist()
wholeyear_index = pd.date_range(start, end)
wholeyear_list = wholeyear_index.strftime("%Y-%m-%d").tolist()

from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holiday_index = cal.holidays(start = "2019-01-01", end = "2019-12-31")
holiday_list = holiday_index.strftime("%Y-%m-%d").tolist() 

wd_list = [i for i in weekday_list if i not in holiday_list]
nwd_list = [i for i in wholeyear_list if i not in wd_list]

No_data_wd = ["2019-12-05","2019-12-06","2019-12-09","2019-12-10","2019-12-31"]
wd_list_effective = [i for i in wd_list if i not in No_data_wd]
wd_list_effective.append("2019-10-14")
wd_list_effective.append("2019-11-11")

# 2019-10-14，2019-11-11大功率
# 2019-01-19，2019-05-27，2019-10-20负数
No_data_nwd = ["2019-01-19","2019-01-20",
               "2019-02-16",
               "2019-04-14","2019-04-28",
               "2019-05-25","2019-05-27",
               "2019-08-17",
               "2019-10-20",
               "2019-11-28","2019-11-30",
               "2019-12-07","2019-12-08","2019-12-22","2019-12-25"]

nwd_list_effective = [i for i in nwd_list if i not in No_data_nwd]
nwd_list_effective = [i for i in nwd_list_effective if i not in ["2019-10-14","2019-11-11"]]

date_linear = ["2019-01-01","2019-03-03","2019-08-31","2019-09-01","2019-12-26"]

# 点太少了，无法拟合出合适的函数，当作异常点处理:"2019-01-01"，"2019-03-03"
# 真的完全服从一条直线，指数函数无法拟合："2019-12-26"，"2019-08-31"，"2019-09-01"
# 服从指数函数的点："2019-01-27","2019-02-02","2019-02-18","2019-07-04","2019-07-05","2019-08-10","2019-08-18","2019-09-08"，
# "2019-10-06"，"2019-11-24","2019-11-29"

wholeyear_effective = [i for i in wholeyear_list if i not in No_data_nwd]
wholeyear_effective = [i for i in wholeyear_effective if i not in No_data_wd]
# len(wholeyear_effective)

In [5]:
# sample one day EV charging data of the workplace from the annual dataset
def sample_EVC_workplace_oneday(EVC_workplace,random_seed):
    """
    EVC_workplace: EV charging data in the workplace for the whole year,df
    random_seed: the value of the random seed for np.random.seed, float
    """
    # EV充电数据含有的日子
    Date_EVC_workplace = (EVC_workplace
                          ["connect_date"]
                          .unique()
                          .tolist())
    
    # EV充电数据含有的工作日、非工作日
    # Date_EVC_workplace_nwd = [i for i in Date_EVC_workplace if i in nwd_list_effective]
    Date_EVC_workplace_wd = [i for i in Date_EVC_workplace if i in wd_list_effective]

    # 随机抽取日期序列，并赋予日期到各个节点中
    np.random.seed(random_seed) # 1 30 40 
    Date_EVC_workplace = np.random.choice(Date_EVC_workplace_wd)
    EVC_workplace = EVC_workplace.query("connect_date == @Date_EVC_workplace").copy().reset_index(drop=True)
    # EVC_workplace.to_csv("./result/EVC/EVC_workplace.csv")
    # EVC_public.to_csv("./result/EVC/EVC_public.csv")
    
    return EVC_workplace 

#  Load flow analysis to calculate the load limit under the constraints on voltage deviation, transformer capacity, and electrical line transmission capacity

In [11]:
# build the grid network model in pandapower
def network_3bus_1tr(p,q,transfo_capacity):
    """
    # the network looks like:
    # ext_grid b1 ---- b2 trafo(10/0.4kV) b3 load

    p: the active power demand, MW, float
    q: the reactive power demand, MVar, float
    trafo_capacity: the sn_mva of the transformer, float
    """ 
    #create empty net
    net = pp.create_empty_network()

    #create buses
    b1 = pp.create_bus(net, vn_kv=10, name="Bus_0")
    b2 = pp.create_bus(net, vn_kv=10, name="Bus_1")
    b3 = pp.create_bus(net, vn_kv=0.4, name="Bus_2")

    #create bus elements
    pp.create_ext_grid(net, bus=b1, name="Grid Connection")
    pp.create_load(net, bus=b3, p_mw=p, q_mvar=q, name='load')

    #create branch elements
    pp.create_line(net,from_bus=b1,to_bus=b2,length_km=0.1,std_type='94-AL1/15-ST1A 10.0',name='line_01')
    if transfo_capacity == 0.63:
        pp.create_transformer(net,hv_bus=b2,lv_bus=b3,name='trafo_12',std_type='0.63 MVA 10/0.4 kV')
    if transfo_capacity == 0.50:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=1.2, pfe_kw=1, i0_percent=0.21, shift_degree=150, name='trafo_12')
    if transfo_capacity == 1.25:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.7, i0_percent=0.14, shift_degree=150, name='trafo_12')
    if transfo_capacity == 1.60:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.8, i0_percent=0.12, shift_degree=150, name='trafo_12')
    if transfo_capacity == 2.00:
        pp.create_transformer_from_parameters(net,hv_bus=b2,lv_bus=b3,sn_mva=transfo_capacity,vn_hv_kv=10,vn_lv_kv=0.4,
                                              vk_percent=4.0, vkr_percent=0.9, pfe_kw=1.9, i0_percent=0.10, shift_degree=150, name='trafo_12')

    return net

In [12]:
# Obtain the maximum loading of the bus under the constraint on the voltage deviation, transformer loading and line loading. 
def func_p_site_max(transfo_capacity,q):
    """
    transfo_capacity: the sn_mva of the transformer, MVA. workplace: 1.25、1.60, public: 0.5、0.63
    q: the reactive load in the bus, MVar
    """
    # section 1: the hyperparameters of obtaining the maximum loading of each bus

    # maximum and minimum bus voltage magnitude unit
    v_maxpu = 1.05
    v_minpu = 0.95
    # the maximum percentage of the transformer loading
    trafo_loading_max = 100
    # the maximum percentage of the line loading
    line_loading_max = 100

    # section 2: conduct calculation

    # the number of charging sites
    num_site = 1
    # the number of bus
    num_bus = 3
    # the number of trafo
    num_trafo = 1
    # the number of line
    num_line = 1
    # Initialize the maximum loading of the bus
    p_site_max = [0 for i in range(num_site)]
    # Assume to enumerate the load by a step of 1e-2 from 0 to 6 mW
    p_values = np.arange(0,6,1e-2)
    for p in p_values: # 模拟每节点的load (mW)
        # run load flow analysis
        net = network_3bus_1tr(p,q,transfo_capacity)
        pp.runpp(net)

        # the voltage deviation of each bus
        v_pu = [net.res_bus.loc[i,'vm_pu'] for i in range(num_bus)]
        # the transformer loading(%)
        trafo_loading = [net.res_trafo.loc[i,'loading_percent'] for i in range(num_trafo)]
        # the line loading (%)
        line_loading = [net.res_line.loc[i,'loading_percent'] for i in range(num_line)]

        # A flag to indicate if any bus has the voltage deviation bigger than the limit. 
        # If yes, return True, the loop breaks; if no, return False, the loop continues.
        # the same for the transformer loading and line loading
        flag_v_pu = any(x < v_minpu or x > v_maxpu for x in v_pu)
        flag_trafo_loading = any(x > trafo_loading_max for x in trafo_loading)
        flag_line_loading = any(x > line_loading_max for x in line_loading)

        # If any flag is true, save the maximum power load for each bus then breaks the loop
        if any((flag_v_pu,flag_trafo_loading,flag_line_loading)):
            for i in range(num_site):
                p_site_max[i] = p
            break
    
    return p_site_max

In [13]:
# Conduct load flow analysis
%%capture
p_site_max_w1600 = func_p_site_max(transfo_capacity = 1.60,q = 0.10) # power factor was set to 0.9

# Functions to conduct online optimization of charging coordination under the computed load limit

In [10]:
# Optimization model 1：Obtain the upper bound of the load variance
def opt_model_ub(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub):
    """
    return:
    m_ub.status: the optimization status of the model
    loadvar_ub: the largest load variance under the constraints of line loading, voltage deviation, and transformer loading, list
    """
    # Create optimization model
    m_ub = gp.Model('EVcharging_ub')

    # Create variables
    # 将字典EV_index转化为a list of tuple，第一位表示site,第二位表示EV的序号
    import itertools
    EV_list = [(k, v) for k, values in EV_index.items() for v in values]
    p_tbi = m_ub.addVars(time, EV_list, name="power") # time行 * num_EV列 * site列的矩阵
    p_t = gp.tuplelist([p_tbi.sum(t,"*","*") for t in time]) # 一个时刻所有site的所有EV的充电功率之和，time*1的向量
    p_bt = gp.tuplelist([[p_tbi.sum(t, site[0], "*") for t in time]])# 一个时刻一个site的所有EV充电功率之和（对EV求和），site*time的矩阵                
    p_tb = gp.tuplelist(list(map(list,zip(*p_bt)))) # 通过转置，变为time*site的矩阵
    p_bi = gp.tuplelist([[p_tbi.sum("*", site[0], i) for i in EV_index[site[0]]]])# 每个site的每辆EV的充电功率之和(对时间求和)
    # p_tb，p_t除了t_now当前时刻，其他都是预测值。当site只有一个时，p_tb等于p_t
#     p_tb_pred[t_now] = p_tb[0][0]
#     p_t_pred = [a for a in p_tb_pred] # [a+b+c for a,b,c in zip(p_tb0_pred, p_tb1_pred, p_tb2_pred)]
#     p_t_mixed = [a for a in p_t_pred]
#     p_t_mixed[t_now] = p_t[t_now-t_now]
     
    # Set objective
    if time[-1] >= num_t:
        MEF = MEF[t_now:] + MEF[:time[-1] - num_t + 1]
    else:
        MEF = MEF[t_now:]
    m_ub.setObjective (sum(np.multiply(p_t,MEF)) * delta_t / ce, GRB.MINIMIZE) # the output is g CO2 emissions after coordination
    
    # Add constraints
    for t in time:
        m_ub.addConstrs(
            (p_tbi[t,s,i] >= 0 for s in site for i in EV_index[s]),name = "charging_power1")
        m_ub.addConstrs(
            (p_tbi[t,s,i] <= p_max for s in site for i in EV_index[s]),name = "charging_power2")
        # p_t, p_bt都是list，引用用序号。第0位序号对应t_now, 第1位序号对应t_now + 1。因此time与序号的关系是t-t_now
        m_ub.addConstrs(
            (p_tb[t-t_now][s]/1_000 + d_t[s][t-t_now]/1_000 <= p_site_max[s] for s in site), name = "site_max_loading")
    for s in site:
        for i in EV_index[s]:
            if t_plugin[s][i] < t_plugout[s][i]:
                if t_plugout[s][i] != time[-1]:
                    m_ub.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,time[-1])),"charging_power_time1")
#             else:
#                 if abs(t_plugin[s][i]-t_plugout[s][i])>1:
#                     m_ub.addConstrs(
#                         (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,int(t_plugin[s][i]))),"charging_power_time1")
    m_ub.addConstrs(
        (ce * delta_t * p_bi[s][i] >= E[s][i] for s in site for i in EV_index[s]),"energy_demand")

    # Conduct optimization
    m_ub.params.NonConvex = 2 # Quadratic equality constraints are non-convex. Set NonConvex parameter to 2 to solve model.
    m_ub.Params.MIPGap = 0.01    # termination criteria: 1%
    m_ub.Params.TimeLimit = 360  # termination criteria: 6 minutes
    m_ub.Params.BarHomogeneous = 0 # turn off the homogeneous barrier algorithm
    m_ub.Params.Method = -1 # automatic method = barrier
    m_ub.update()
    m_ub.optimize()
    if m_ub.status == GRB.OPTIMAL:
        solution_p_ub = m_ub.getAttr('x', p_tbi)
        loadvar_ub_value = sum((a + b) ** 2 for a, b in zip(D_t, [solution_p_ub.sum(t, '*','*').getValue() for t in time[:num_t - t_now + 1]]))
        loadvar_ub.append(loadvar_ub_value)
        
    return m_ub.status

In [11]:
# Optimization model 2：Optimize under the constraint on the load varaince
def opt_model(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub,p_tb_cc_0,p_tnowbi):
    """
    return:
    m.status: the optimization status of the model
    p_tb_cc_0[t_now]: the total charging load of the chaging EVs at t_now, float
    p_tnowbi: the charging load of the charging EVs at t_now, list
    """
    # Create optimization model
    m = gp.Model('EVcharging')

    # Create variables
    # 将字典EV_index转化为a list of tuple，第一位表示site,第二位表示EV的序号
    import itertools
    EV_list = [(k, v) for k, values in EV_index.items() for v in values]
    p_tbi = m.addVars(time, EV_list, name="power") # time行 * num_EV列 * site列的矩阵
    p_t = gp.tuplelist([p_tbi.sum(t,"*","*") for t in time]) # 一个时刻所有site的所有EV的充电功率之和，time*1的向量
    p_bt = gp.tuplelist([[p_tbi.sum(t, site[0], "*") for t in time]])# 一个时刻一个site的所有EV充电功率之和（对EV求和），site*time的矩阵                
    p_tb = gp.tuplelist(list(map(list,zip(*p_bt)))) # 通过转置，变为time*site的矩阵
    p_bi = gp.tuplelist([[p_tbi.sum("*", site[0], i) for i in EV_index[site[0]]]])# 每个site的每辆EV的充电功率之和(对时间求和)
    
    # Set objective
    if time[-1] >= num_t:
        MEF = MEF[t_now:]+MEF[:time[-1] - num_t + 1]
    else:
        MEF = MEF[t_now:]
    m.setObjective (sum(np.multiply(p_t,MEF)) * delta_t / ce, GRB.MINIMIZE) # the output is g CO2 emissions after coordination

    # Add constraints
    for t in time:
        m.addConstrs(
            (p_tbi[t,s,i] >= 0 for s in site for i in EV_index[s]),name="charging_power1")
        m.addConstrs(
            (p_tbi[t,s,i] <= p_max for s in site for i in EV_index[s]),name="charging_power2")
        # p_t, p_bt都是list，引用用序号。第0位序号对应t_now, 第1位序号对应t_now + 1。因此time与序号的关系是t-t_now
        m.addConstrs(
            (p_tb[t-t_now][s]/1_000 + d_t[s][t-t_now]/1_000 <= p_site_max[s] for s in site), name = "site_max_loading")
    for s in site:
        for i in EV_index[s]:
            if t_plugin[s][i] < t_plugout[s][i]:
                if t_plugout[s][i] != time[-1]:
                    m.addConstrs(
                        (p_tbi[t,s,i] == 0 for t in range(int(t_plugout[s][i])+1,time[-1])),"charging_power_time1")
    m.addConstrs(
        (ce * delta_t * p_bi[s][i] >= E[s][i] for s in site for i in EV_index[s]),"energy_demand")
    m.addConstr(
    (sum((a + b) ** 2 for a, b in zip(D_t, [p_tbi.sum(t,"*","*") for t in time[:num_t - t_now + 1]])) <= loadvar_ub[0]),name="load_variance")
   
    # Conduct optimization
    m.params.NonConvex = 2 # Quadratic equality constraints are non-convex. Set NonConvex parameter to 2 to solve model.
    m.Params.MIPGap = 0.01    # termination criteria: 1%
    m.Params.TimeLimit = 360  # termination criteria: 6 minutes
    m.Params.BarHomogeneous = 0 # turn off the homogeneous barrier algorithm
    m.Params.Method = -1 # automatic method = barrier
    m.update()
    m.optimize()
    if m.status == GRB.OPTIMAL:
        solution_p = m.getAttr('x', p_tbi)
#         p_tb_cc_0[t_now] = solution_p.sum(t_now,0,'*').getValue()
#         for i in EV_index[site[0]]:
#             p_tnowbi.append(solution_p[t_now,0,i])            
        for i in EV_index_real[site[0]]:    
#             if t_now == t_plugout[0][i]: # 如果是EV插入的最后一个时段了，就把它剩余能量充满。
#                 # 考虑不充分。如果剩余能量太多，可能充电桩的瞬时功率就无法满足或者造成电网过载。
#                 solution_p[t_now,0,i] = E[0][i]/(ce * delta_t)
            p_tnowbi.append(solution_p[t_now,0,i]) # 应该只加EV_do的real的部分
        p_tb_cc_0[t_now] = sum([solution_p[t_now,0,i] for i in EV_index_real[site[0]]])

    return m.status

In [12]:
# Function to get the EVC_prediced 获取days-ahead的EVC均值
def func_EVC_sta(EVC_site_0,EVC_annual,len_dates_to_sta):
    """
    Param:
    EVC_site_0: EV charging session data of a specific date, df
    EVC_annual: EV charging session data for the whole year, df
    len_dates_to_sta: the numbers of dates for the statistics, float
    
    Return:
    EVC_sta: the df specifies the average number of EVs arriving at each timeslot, 
             the average departure time of EVs arriving at each timeslot, 
             the average required energy of EVs arriving at each timeslot for , df
             
    从EVC_site_0读出日期，然后从EVC_annual获得EVC_sta，根据EVC_sta生成EVC数据
    """
    # copy an imported df
    df_copy = EVC_site_0.copy()
    df_annual = EVC_annual.copy()
    
    # Convert the 'connect_date' column to a datetime object
    df_copy['connect_date'] = pd.to_datetime(df_copy['connect_date'])
    df_annual['connect_date'] = pd.to_datetime(df_annual['connect_date'])
    
    # extract the date of EVC_site_0
    input_date = df_copy['connect_date'][0]
    
    # Calculate the 7 days before the input date. The dates are in pd.timestamps
    result_date = [input_date - pd.DateOffset(days=i) for i in range(1,len_dates_to_sta+1)]
    
    # Use the result_date to filter the EVC from df_annual
    df_annual_sta = df_annual[df_annual['connect_date'].isin(result_date)]
    
    # Calculate daily averages
    # 各时刻每天平均到达车辆数 Daily Average Number of EVs Arriving at Each Timeslot
    daily_avg_arrival = (df_annual_sta
                         .groupby(['connect_date','connectionTime_index'])
                         ['connectionTime_index'].count()
                         .groupby('connectionTime_index').mean()
                         .round())
    # 各个时刻到达的车辆每辆车的平均离开时刻，取整. Average Departure Time of each EV Arriving at Each Timeslot, rounded
    avg_departure = (df_annual_sta
                     .groupby(['connectionTime_index'])
                     ['disconnectTime_index'].mean().
                     round())
    # 各个时刻到达的车辆每辆车的平均能量需求, Average Required Energy for each EV Arriving at Each Timeslot
    avg_energy = (df_annual_sta
                  .groupby(['connectionTime_index'])
                  ['kWhDelivered'].mean())
    # output the result
    EVC_sta = pd.DataFrame(index=range(96))
    EVC_sta = pd.concat([EVC_sta,daily_avg_arrival,avg_departure,avg_energy],axis=1)
    EVC_sta.columns = ['num_arrival_avg','disconnectTime_index_avg','kWhDelivered_avg']
    
    return EVC_sta.reset_index(drop=True)

def func_EVC_pred(t,EVC_sta):
    """
    Param:
    t: the timeslot starting to have the predicted EVC, t = t_now + 1, int
    EVC_sta: the df specifies the average number of EVs arriving at each timeslot, 
             the average departure time of EVs arriving at each timeslot, 
             the average required energy of EVs arriving at each timeslot, df
    
    Return:
    EVC_pred: the predicted EVC from t_now to the end of optimization horizon, df
    """
    # extract the statistic data according to t_now, and drop the null value, then copy the df
    EVC_sta_do = (EVC_sta
                  .loc[t:,:]
                  .dropna()
                  .copy())
    
    # create an EVC_pred df to store the created EV charging sessions based on EVC_sta_do    
    EVC_pred = pd.DataFrame(columns = ['connectionTime_index','disconnectTime_index','kWhDelivered'])
    
    # create EV charging sessions for EVC_pred
    for idx in EVC_sta_do.index:
        num_EV = int(EVC_sta_do.loc[idx,'num_arrival_avg'])
        for i in range(num_EV):
            EVC_session = {
                'connectionTime_index':[idx],
                'disconnectTime_index':[EVC_sta_do.loc[idx,'disconnectTime_index_avg']],
                'kWhDelivered':[EVC_sta_do.loc[idx,'kWhDelivered_avg']]
            }
            EVC_session = pd.DataFrame(EVC_session)
            EVC_pred = pd.concat([EVC_pred,EVC_session],axis = 0)
    
    EVC_pred['type'] = 'pred'
    
    return EVC_pred.reset_index(drop=True)

In [14]:
# Function to conduct charging coordination for the whole day
def func_opt(d_t_site_0,EVC_site_0,EVC_annual,p_site_max,MEF,random_seed):
    """
    param:
    d_t_site_0: base load data (kW) in every 15 minutes, df
    EVC_site_0: EV charging session data (kW), df
    EVC_annual: EV charging session data for the whole year, df
    p_site_max: the maximum loading (MW) of the bus under the constraint on the voltage deviation,transformer loading and line loading,list
    MEF: the marginal emission factors of each hour of a day, list
    random_seed: the random seed to label the EVC_site_0, float
    
    output: 
    obj_result, storing the result of Em_uc, Em_cc,Em_reduce,loadvar_uc,loadvar_cc,loadvar_reduce, df
    p_tb_cc: the total charging load at each time t in coordinated charging, the indexes are the timeslot, df
    p_tb_uc: the total charging load at each time t in uncoordinated charging, the indexes are the timeslot, df
    """
    # section 1: base data    
    
    # time interval (hour)
    delta_t = 0.25
    
    # number of time intervals of a day
    num_t = int(24 * 1 / delta_t)
    time_full = [i for i in range(0,num_t)]

    # EV charging efficiency
    ce = 0.9

    # charging power limit of the L2 charger(kW)
    p_max = 14 # 10
    
    # coefficient of the uncoordinated EV charging load
    # alpha = [i/10_000 for i in range(10_000,8_992,-8)]
    # alpha = 0.93
    
    # add a -5% to 5% noise to the real base load to act as the predicted base load
    d_t_site_0_real = d_t_site_0.iloc[:,0].tolist()
    np.random.seed(random_seed)
    noise = np.random.uniform(-0.05,0.05,num_t)
    d_t_site_0_pred = [a * (1 + b) for a,b in zip(d_t_site_0.iloc[:,0].tolist(),noise)]
#     d_t_site_0_pred = d_t_site_0.iloc[:,0].tolist()
    
    # obtain the 7 days-ahead EVC statistics
    len_dates_to_sta = 7
    EVC_sta = func_EVC_sta(EVC_site_0, EVC_annual, len_dates_to_sta)
    
#     # turn the cross-day disconnectTime_index to be larger than 95 and calculate the unit energy demand of each charging session
#     for i in EVC_site_0.index:
#         if EVC_site_0.loc[i,'connectionTime_index'] >= EVC_site_0.loc[i,'disconnectTime_index']:
#             EVC_site_0.loc[i,'disconnectTime_index'] += num_t
#         EVC_site_0.loc[i,'kWhDelivered_unit'] = EVC_site_0.loc[i,'kWhDelivered'] / (EVC_site_0.loc[i,'disconnectTime_index'] - EVC_site_0.loc[i,'connectionTime_index'] + 1)            
#         if EVC_site_0.loc[i,'connectionTime_index'] >= EVC_site_0.loc[i,'doneChargingTime_index']:
#             EVC_site_0.loc[i,'doneChargingTime_index'] += num_t
#         EVC_site_0.loc[i,'charging_power'] = EVC_site_0.loc[i,'kWhDelivered'] / ((EVC_site_0.loc[i,'doneChargingTime_index'] - EVC_site_0.loc[i,'connectionTime_index'] + 1) * delta_t * ce)
    
    # calculate the unit energy demand of each charging session
    for i in EVC_site_0.index: 
        EVC_site_0.loc[i,'kWhDelivered_unit'] = EVC_site_0.loc[i,'kWhDelivered'] / (EVC_site_0.loc[i,'disconnectTime_index'] - EVC_site_0.loc[i,'connectionTime_index'] + 1)                 
            
    # calculate the total energy requirement of EVC
    total_energy = 0
    for i in EVC_site_0.index:
        if EVC_site_0.loc[i,"doneChargingTime_index"] > num_t - 1:
            total_energy += EVC_site_0.loc[i,'kWhDelivered'] * (num_t - EVC_site_0.loc[i,'connectionTime_index']) / (EVC_site_0.loc[i,'doneChargingTime_index'] - EVC_site_0.loc[i,'connectionTime_index'] + 1)    
        else:    
            total_energy += EVC_site_0.loc[i,'kWhDelivered']
    
    # add the real label to EVC_site_0
    EVC_site_0['type'] = 'real'
    
    # section 2: Obtain the total load(kW), load variance, and CO2 emissions(kg) in uncoordinated charging
    
    # Obtain the charging demand (kW) at 15-min resolution
    def func_demand_load(EVC,num_t):
        """
        this func is used to calculate the charging load of a day in every 15 minutes
        param: 
        EVC: daily EV charging load, df
        num_t: number of time intervals of a day
        return:
        demand_load: the charging demand (kW) in every 15 minutes, list
        """
        # df for the total charging load in every 15 minutes(kW)
        demand_load = pd.DataFrame(data = [0 for i in range(0,num_t)])
        
        # copy EVC for revise
        df = EVC.copy()

        # load的第二列赋值，即计算某日EV充电负载，利用EV_charging_data按车作循环
        for i in df.index: # i是EV charging data的行，表示第i辆EV
            # 获得开始和结束的充电时间
            start_time = df.loc[i,"connectionTime_index"]
            end_time = df.loc[i,"doneChargingTime_index"] 
            power = df.loc[i,"charging_power"]
            if end_time >= num_t:
                for j in range(int(start_time),num_t,1):
                    demand_load.iloc[j,0] += power
            else:
                for j in range(int(start_time),int(end_time)+1,1):
                    demand_load.iloc[j,0] += power
        
        return demand_load[0].tolist()
    p_tb_demand = func_demand_load(EVC = EVC_site_0,num_t = num_t)
    p_tb_uc = [] # set up a list to store the power demand (kW) of uncoordinated charging at every 15-min
    for t in time_full:
        if p_tb_demand[t] + d_t_site_0_real[t] > p_site_max[0] * 1_000:
            p_tb_uc.append(p_site_max[0] * 1_000 - d_t_site_0_real[t])
        else:
            p_tb_uc.append(p_tb_demand[t])
    loadvar_uc = sum((a + b) ** 2 for a, b in zip(d_t_site_0_real,p_tb_uc))
    
    # Obtain the total CO2 emissions by inputting the EV charigng load data
    def func_em(MEF_data,EV_charging_load,ce):
        """
        This function compute the CO2 emissions in uncoordinated charging    
        param 
        MEF_data: the MEF (g CO2/kWh) in every 15 mins of a day,list
        EV_charging_load: EV charging load (kW) before coordination in every 15 mins of a day, list
        ce: charging efficiency (%)
        return
        ucem: the total CO2 emissions in uncoordianted charging，array，unit: kg
        """
        # 利用两个向量内积，计算总排放量，充电效率设为0.85。时间由15分钟转换为小时，即乘以0.25
        ucem = (np.dot([0.25 * x for x in EV_charging_load],MEF_data)/ce)/1_000 # 将碳排放数据从g转换为kg

        return ucem
    Em_uc = (func_em(MEF_data = MEF, EV_charging_load = p_tb_uc, ce = ce))/(sum(p_tb_uc)*delta_t*ce) # kg/kWh
    
    # 预测每个时刻迎来的充电需求，但未做分配（非协调充电，扣除该时刻，各个时刻的充电需求，然后和该时刻真实的充电需求一起做协调充电）。
    # 非协调充电时的各个时刻的charging demand曲线。（不知道有多少辆车，也不知道啥时候走，这些都会有历史数据的啊。各个时刻平均来多少辆车，他们平均啥时候离开）
    # 说到底，就是这个时刻的历史数据，包括来多少辆车，平均每辆车需要的能量，平均离开时间..
    # 所以我要输入的，是历史平均的EVC数据表，index时刻，列名平均到达车辆数，每辆车辆平均离开时刻，每辆车辆平均能量需求
    # 统计的时间尺度可以是年、月的统计量。
    # 然后现实协调充电时，就拿该时刻真实的充电需求，和其他时刻基于历史统计的充电需求数据一起做协调优化。
    # 所以我要变的是EVC，并不是优化模型。优化模型和off-line是一样的。
    # EVC动态变化，time动态变化。
    
    # Section 3: conduct charging coordination under the constraint of p_site_max
    
    # Initialize the t_now to conduct the loop
    p_tb_cc_0 = [0 for i in range(0,num_t)]
    for t_now in time_full:        
        # extract the EVC for coordination at t_now
        EVC_real = EVC_site_0.query("(connectionTime_index <= @t_now) & (disconnectTime_index >= @t_now) & (kWhDelivered > 0)").copy()        
        # if EVC_real is not empty, it means there are EV sessions that are needed to be optimized
        Flag = EVC_real.empty
        if Flag == False:            
            EVC_pending = EVC_site_0[~EVC_site_0.index.isin(EVC_real.index)].copy().reset_index(drop=True)
            EVC_tem = pd.DataFrame(columns = EVC_real.columns) # the dataframe to temporarily save the discard charging session from EVC_do
            EVC_pred = func_EVC_pred(t_now + 1,EVC_sta)
#             EVC_pred = EVC_site_0.query("connectionTime_index > @t_now").copy() # 当预测数据为真实数据时
#             EVC_pred['type'] = 'pred'
            EVC_do = pd.concat([EVC_real,EVC_pred],axis = 0,ignore_index = True)
            # the dynamic optimization horizon                
            max_time = int(EVC_do.disconnectTime_index.max())
            if max_time > num_t:
                time = [i for i in range(t_now,max_time)]
            else:
                time = [i for i in range(t_now,num_t)]
            # d_t_site_0是实际值加预测值, t_now is the real number, others are the predicted ones, 动态变化
            if time[-1] == num_t - 1: # 如果time的范围等于96，d_t不用延伸
                d_t_site_0_mixed = d_t_site_0_pred[t_now:]
            else: # 如果time的范围大于96，d_t需要延伸到（time - 96）+ 1（因为最后一个取不了）
                d_t_site_0_mixed = d_t_site_0_pred[t_now:] + d_t_site_0_pred[:time[-1] - num_t + 1]
            d_t_site_0_mixed[0] = d_t_site_0_real[t_now]
            # the multidict indexed by the charging site
            site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout = gp.multidict({
                0:[d_t_site_0_mixed, # the based load
                   EVC_do.index, # the index of EVs at each bus
                   EVC_do.query("type == 'real'").index, # the index of EVs at present at each bus
                   EVC_do['kWhDelivered'].tolist(), # the charging energy requirement of each EV at each bus, unit: kWh
                   EVC_do['connectionTime_index'].tolist(),# the plug-in time of EVs at each bus (index of every 15 minutes)
                   EVC_do['disconnectTime_index'].tolist()] # the plug-out time of EVs at each bus (index of every 15 minutes)
            })
            # total base load (kW)
            D_t = [a for a in d_t[0]]
            # initialize the loadvar_ub
            loadvar_ub = []
            # obtain the upper bound(largest) of the load variance
            state_ub = opt_model_ub(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,
                                    site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub)
            if state_ub != GRB.OPTIMAL:
                # sort the EVC_do with unit energy demands in an ascending order       
                EVC_do = EVC_do.sort_values(by='kWhDelivered_unit').reset_index(drop=True)
                Num_EVC_do = EVC_do.shape[0] # the number of EV charging sessions in EVC_do before dropping
                for i in range(Num_EVC_do):
                    if EVC_do.shape[0] >= 1:
                        if EVC_do.loc[0,'type'] == 'real':
                            new_row = EVC_do.loc[0].to_frame().T # convert the Series to DataFrame with the to_frame() method and transposing it with T, you get a one-row DataFrame.
                            EVC_tem = pd.concat([EVC_tem,new_row],axis = 0,ignore_index = True)
                        EVC_do = EVC_do.drop(0).reset_index(drop=True)
                        site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout = gp.multidict({
                            0:[d_t_site_0_mixed, # the based load
                               EVC_do.index, # the index of EVs at each bus
                               EVC_do.query("type == 'real'").index, # the index of EVs at present at each bus
                               EVC_do['kWhDelivered'].tolist(), # the charging energy requirement of each EV at each bus, unit: kWh
                               EVC_do['connectionTime_index'].tolist(),# the plug-in time of EVs at each bus (index of every 15 minutes)
                               EVC_do['disconnectTime_index'].tolist()] # the plug-out time of EVs at each bus (index of every 15 minutes)
                        })
                        state_ub = opt_model_ub(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,
                                                site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub)
                        if state_ub == GRB.OPTIMAL:
                            break
#             print(f'loadvar_ub is {loadvar_ub[0]}')
            # initialize the p_tnowbi
            p_tnowbi = []
            # optimization with load variance
            state = opt_model(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,
                              site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub,p_tb_cc_0,p_tnowbi)
            if state != GRB.OPTIMAL:
                if state_ub == GRB.OPTIMAL: # if opt_ub能第一次就有最优解，那么EVC_do就还没有排序，Num_EVC_do也还没有
                    EVC_do = EVC_do.sort_values(by='kWhDelivered_unit').reset_index(drop=True)
                    Num_EVC_do = EVC_do.shape[0] # the number of EV charging sessions in EVC_do before dropping   
                for i in range(Num_EVC_do):
                    if EVC_do.shape[0] >= 1:
                        if EVC_do.loc[0,'type'] == 'real':
                            new_row = EVC_do.loc[0].to_frame().T 
                            EVC_tem = pd.concat([EVC_tem,new_row],axis = 0,ignore_index = True)
                        EVC_do = EVC_do.drop(0).reset_index(drop=True)
                        site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout = gp.multidict({
                            0:[d_t_site_0_mixed, # the based load
                               EVC_do.index, # the index of EVs at each bus
                               EVC_do.query("type == 'real'").index, # the index of EVs at present at each bus
                               EVC_do['kWhDelivered'].tolist(), # the charging energy requirement of each EV at each bus, unit: kWh
                               EVC_do['connectionTime_index'].tolist(),# the plug-in time of EVs at each bus (index of every 15 minutes)
                               EVC_do['disconnectTime_index'].tolist()] # the plug-out time of EVs at each bus (index of every 15 minutes)
                        })
                        state = opt_model(t_now,num_t,time,ce,p_max,delta_t,p_site_max,D_t,MEF,
                                          site,d_t,EV_index,EV_index_real,E,t_plugin,t_plugout,loadvar_ub,p_tb_cc_0,p_tnowbi)
                        if state == GRB.OPTIMAL:
                            break
            
#             print(f't_now is {t_now}')
#             p_tnowbi_index = []
#             for idx, x in enumerate(p_tnowbi):
#                 p_tnowbi_index.append(idx)   
#             print(f'the index of p_tnowbi is {p_tnowbi_index}')
#             EV_index_index = []
#             for idx, x in enumerate(EV_index[0]):
#                 EV_index_index.append(idx)  
#             print(f'the index of EV is {EV_index_index}')
            
            # depart the predicted EVC_do and the real EVC_do       
            EVC_real = EVC_do.query("type == 'real'")
            # update energy demand and unit of energy demand in the EVC_do
            for i in EVC_real.index:
                EVC_real.loc[i,'kWhDelivered'] -= p_tnowbi[i] * delta_t * ce
                if t_now < EVC_real.loc[i,'disconnectTime_index']: # EV插入的最后一个时段就无需计算kWhDelivered_unit了
                    EVC_real.loc[i,'kWhDelivered_unit'] = EVC_real.loc[i,'kWhDelivered']/(EVC_real.loc[i,'disconnectTime_index'] - t_now + 1)
#                 EVC_real.loc[i,'connectionTime_index'] += 1
            # concat the dataframe of the updated EVC_do, EVC_pending, EVC_tem as the EVC_site_0 to conduct the next interation
            EVC_site_0 = pd.concat([EVC_real,EVC_pending,EVC_tem],axis = 0,ignore_index = True)
            # the kWhDelivered of the final EVC_do represent the remaining energy demand that has not been satisfied during the plug-in period
            
    # Section 4: output the final result
    p_tb_cc = [a for a in p_tb_cc_0]
    Em_cc = (func_em(MEF_data = MEF,EV_charging_load = p_tb_cc,ce = ce))/(sum(p_tb_cc)*delta_t*ce) # kg/kWh
    # obtain the reduction in load variance and emissions
    loadvar_cc = sum((a + b) ** 2 for a, b in zip(d_t_site_0_real, p_tb_cc))
    loadvar_reduce = ((loadvar_uc - loadvar_cc)/loadvar_uc) * 100
    Em_reduce_percent = ((Em_uc - Em_cc) / Em_uc) * 100
    # obtain the percentage of the satisfied energy requirement
    EVC_site_0['kWhDelivered'][EVC_site_0['kWhDelivered'] < 1e-3] = 0 
    satisfied_energy = (1 - EVC_site_0.kWhDelivered.sum() / total_energy) * 100
    # obtain the total saving on emissions (kg)
    Em_reduce = (Em_uc - Em_cc) * (satisfied_energy / 100) * total_energy
    # create a df to store the results of Em_uc, Em_cc,Em_reduce,loadvar_uc,loadvar_cc,loadvar_reduce
    obj_result = pd.DataFrame(index = [random_seed],columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])      
    obj_result.loc[random_seed,'Em_uc_kg/kWh'] = Em_uc # kg/kWh
    obj_result.loc[random_seed,'Em_cc_kg/kWh'] = Em_cc # kg/kWh
    obj_result.loc[random_seed,'Em_reduce_%'] = Em_reduce_percent # %
    obj_result.loc[random_seed,'Em_reduce_kg'] = Em_reduce # kg
    obj_result.loc[random_seed,'loadvar_uc'] = loadvar_uc
    obj_result.loc[random_seed,'loadvar_cc'] = loadvar_cc
    obj_result.loc[random_seed,'loadvar_reduce_%'] = loadvar_reduce # %
    obj_result.loc[random_seed,'satisfied_energy_%'] = satisfied_energy # %
    obj_result.loc[random_seed,'satisfied_energy_uc_%'] = (sum(p_tb_uc)/sum(p_tb_demand)) * 100 # %
    
    # transfer p_tb_cc and p_tb_uc from a list to a df
    p_tb_cc = pd.DataFrame(p_tb_cc)
    p_tb_uc = pd.DataFrame(p_tb_uc)
    
    return obj_result, p_tb_cc, p_tb_uc

# Conduct the functions and output the result

In [14]:
# Define the DataFrames to store the outputs for the level 1 of annual existing electrical load

# For pareto front
df_pareto_w1600_450_25th = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of uncontrolled charging
df_p_tb_cc_w1600_450_25th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40)])

# For the charging load of coordinated charging
df_p_tb_uc_w1600_450_25th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50,random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50, p_tb_uc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_25th, EVC_site_0 = EVC_workplace_50,
                                                                   EVC_annual = EVC_workplace_annual_50, p_site_max = p_site_max_w1600,
                                                                   MEF = MEF, random_seed = i)
    df_pareto_w1600_50_25th.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50_25th[i] = p_tb_cc_w1600_50
    df_p_tb_uc_w1600_50_25th[i] = p_tb_uc_w1600_50

In [16]:
# Output the result
df_pareto_w1600_50_25th.to_csv("./result/pareto_oncc/workplace/1600KVA/pareto_w1600_50_week_25th.csv")
df_p_tb_cc_w1600_50_25th.to_csv("./result/EV_charging_load_oncc/workplace/1600KVA/p_tb_cc_w1600_50_week_25th.csv")
df_p_tb_uc_w1600_50_25th.to_csv("./result/EV_charging_load_uc/workplace/1600KVA/p_tb_uc_w1600_50_25th.csv")

In [25]:
# Define the DataFrames to store the outputs for the level 2 of annual existing electrical load

# For pareto front
df_pareto_w1600_50 = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of uncontrolled charging
df_p_tb_cc_w1600_50 = pd.DataFrame(index = range(0,96),columns = [i for i in range(40,70)])

# For the charging load of coordinated charging
df_p_tb_uc_w1600_50 = pd.DataFrame(index = range(0,96),columns = [i for i in range(40,70)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50,random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50, p_tb_uc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_avg, EVC_site_0 = EVC_workplace_50,
                                                               EVC_annual = EVC_workplace_annual_50, p_site_max = p_site_max_w1600,
                                                               MEF = MEF, random_seed = i)
    df_pareto_w1600_50.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50[i] = p_tb_cc_w1600_50
    df_p_tb_uc_w1600_50[i] = p_tb_uc_w1600_50

In [ ]:
# Output the result
df_pareto_w1600_50.to_csv("./result/pareto_oncc/workplace/1600KVA/pareto_w1600_50_week.csv")
df_p_tb_cc_w1600_50.to_csv("./result/EV_charging_load_oncc/workplace/1600KVA/p_tb_cc_w1600_50_week.csv")
df_p_tb_uc_w1600_50.to_csv("./result/EV_charging_load_uc/workplace/1600KVA/p_tb_uc_w1600_50.csv")

In [15]:
# Define the DataFrames to store the outputs for the level 3 of annual existing electrical load

# For pareto front
df_pareto_w1600_50_75th = pd.DataFrame(index = range(40,70),columns = ['Em_uc_kg/kWh','Em_cc_kg/kWh','Em_reduce_%','loadvar_uc','loadvar_cc','loadvar_reduce_%','satisfied_energy_%','satisfied_energy_uc_%'])

# For the charging load of uncontrolled charging
df_p_tb_cc_w1600_50_75th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40)])

# For the charging load of coordinated charging
df_p_tb_uc_w1600_50_75th = pd.DataFrame(index = range(0,96),columns = [i for i in range(40)])

In [ ]:
# Conduct the simulation for 30-days
%%capture
for i in range(40,70):
    EVC_workplace_50 = sample_EVC_workplace_oneday(EVC_workplace_annual_50,random_seed = i)
    pareto_w1600_50, p_tb_cc_w1600_50, p_tb_uc_w1600_50 = func_opt(d_t_site_0 = BL_workplace_2019_75th, EVC_site_0 = EVC_workplace_50,
                                                                   EVC_annual = EVC_workplace_annual_50, p_site_max = p_site_max_w1600,
                                                                   MEF = MEF, random_seed = i)
    df_pareto_w1600_50_75th.loc[i,:] = pareto_w1600_50.loc[i,:]
    df_p_tb_cc_w1600_50_75th[i] = p_tb_cc_w1600_50
    df_p_tb_uc_w1600_50_75th[i] = p_tb_uc_w1600_50

In [17]:
# Output the result
df_pareto_w1600_50_75th.to_csv("./result/pareto_oncc/workplace/1600KVA/pareto_w1600_50_week_75th.csv")
df_p_tb_cc_w1600_50_75th.to_csv("./result/EV_charging_load_oncc/workplace/1600KVA/p_tb_cc_w1600_50_week_75th.csv")
df_p_tb_uc_w1600_50_75th.to_csv("./result/EV_charging_load_uc/workplace/1600KVA/p_tb_uc_w1600_50_75th.csv")